In [11]:
import pandas as pd
from typing import List, Dict, Any

class SymptomInput:
    ALLOWED_SYMPTOMS = {
        "fever", "cough", "chest pain", "shortness of breath", "headache", "abdominal pain",
        "vomiting", "diarrhea", "fatigue", "rash", "dizziness", "sore throat", "loss of taste"
    }

    @staticmethod
    def validate(symptoms: List[str]) -> List[str]:
        """Clean and validate symptoms."""
        return [s.lower().strip() for s in symptoms if s.lower().strip() in SymptomInput.ALLOWED_SYMPTOMS]

    @staticmethod
    def from_string(symptoms_str: str) -> List[str]:
        """Convert comma-separated symptom string to list."""
        return SymptomInput.validate([x for x in symptoms_str.split(',')])

class RiskAssessmentEngine:
    SEVERITY_RULES = {
        "chest pain": 3,
        "shortness of breath": 3,
        "dizziness": 2,
        "fever": 1,
        "headache": 1,
        "cough": 1,
        "abdominal pain": 2,
        "vomiting": 2,
        "diarrhea": 1,
        "fatigue": 1,
        "rash": 1,
        "sore throat": 1,
        "loss of taste": 1,
    }

    @staticmethod
    def calculate_score(symptoms: List[str], age: int = None, duration_days: int = None) -> int:
        score = sum(RiskAssessmentEngine.SEVERITY_RULES.get(symptom, 0) for symptom in symptoms)
        if age is not None and (age < 5 or age > 70):
            score += 1
        if duration_days is not None and duration_days > 7:
            score += 1
        return score

    @staticmethod
    def risk_level(score: int) -> str:
        if score >= 5:
            return "High"
        elif score >= 3:
            return "Moderate"
        else:
            return "Low"

    @staticmethod
    def assess(symptoms: List[str], age: int = None, duration_days: int = None) -> Dict[str, Any]:
        score = RiskAssessmentEngine.calculate_score(symptoms, age, duration_days)
        risk = RiskAssessmentEngine.risk_level(score)
        return {"score": score, "risk": risk}

class RecommendationEngine:
    @staticmethod
    def recommend(risk: str, symptoms: List[str]) -> str:
        if risk == "High":
            if "chest pain" in symptoms or "shortness of breath" in symptoms:
                return "Seek emergency care immediately (call 911 or visit ER)."
            return "Consult a healthcare provider as soon as possible."
        elif risk == "Moderate":
            return "Schedule a visit with your healthcare provider within the next 24-48 hours."
        else:
            return "Self-care at home. Monitor your symptoms. If they worsen, seek medical advice."

def format_output(symptoms: List[str], assessment: Dict[str, Any], recommendation: str) -> str:
    return (
        f"Symptoms reported: {', '.join(symptoms) or 'None'}\n"
        f"Risk level: {assessment['risk']} (Score: {assessment['score']})\n"
        f"Recommended action: {recommendation}\n"
    )


def check_symptoms(symptoms_input, age: int = None, duration_days: int = None) -> str:
    if isinstance(symptoms_input, str):
        symptoms = SymptomInput.from_string(symptoms_input)
    elif isinstance(symptoms_input, list):
        symptoms = SymptomInput.validate(symptoms_input)
    else:
        symptoms = []
    assessment = RiskAssessmentEngine.assess(symptoms, age, duration_days)
    recommendation = RecommendationEngine.recommend(assessment["risk"], symptoms)
    return format_output(symptoms, assessment, recommendation)

def run_on_dataset(csv_path="/content/symptom_checker_dataset.csv", output_path="output_results.csv"):
    df = pd.read_csv(csv_path)
    results = []
    for _, row in df.iterrows():
        symptoms = row["Symptoms"]
        age = int(row["Age"])
        duration = int(row["Duration_Days"])
        result = check_symptoms(symptoms, age, duration)
        results.append(result)
    df["AI_Recommendation"] = results
    df.to_csv(output_path, index=False)
    print(f"Results written to {output_path}")


# ------------- Example -------------
if __name__ == "__main__":
    print(check_symptoms("chest pain, fever, cough", age=45, duration_days=2))


Symptoms reported: chest pain, fever, cough
Risk level: High (Score: 5)
Recommended action: Seek emergency care immediately (call 911 or visit ER).

